# Generate Text-Fabric Resource from Source Texts

## Text preprocessing for Christian Urmi and Barwar texts


In order to work with the language data in the NENA corpus, it is important to separate the language text from other data, such as titles, authors/informants, and verse numbers.

Our version of the text comes in MS-Word document files. That is probably also the version of the text that is the richest in language data. It contains not only the text itself, but also meaningful formatting, e.g. word markers set in superscript, and foreign (loan) words set in roman type (where the regular text is set in italic type).

To convert that information from a Word document to something that we can use in Python, we first convert the word documents to HTML, using LibreOffice in headless mode. It is assumed that the Word files are in the subdirectory `texts`, where the converted `.html` files will also be saved.

    $ soffice --headless --convert-to html texts/*.doc

This produces HTML 4.0 documents in the same directory. Earlier attempts with XHTML using wvWare/AbiWord, or LibreOffice using the XHTML conversion filter, produced output that was more difficult to parse or lacked certain characters that were lost in conversion. Although the conversion with LibreOffice takes a very long time compared with AbiWord, the resulting text seems more reliable.

The custom `nena_corpus` package contains the `Text` class, and several functions that assist in the conversion from HTML, of which we only need the functions `html_to_text()` and `parse_metadata()`.

The function `html_to_text()` is a generator function yielding `Text` objects, each containing one paragraph of text.

The function `parse_metadata()` extracts metadata from heading paragraphs (e.g. `title`, `text_id`, `informant`, `place`).

The `Text` class contains an attribute `type` describing the type of paragraph (e.g., `'sectionheading'`, `'p'`, or `'footnote'`), and a list of tuples, containing the text and text style. A text like `'<i>Normal, </i>foreign<i>, and normal</i>'` becomes `[('Normal, ', ''), ('cursive,', 'italic'), (' and normal', '')]` (note the inversion -- because normal text in the source is actually set in italics).

`Text` objects are iterable. New items can be appended with the `append(text, text_style)` method.

In [1]:
from nena_corpus import Text, html_to_text, parse_metadata

A small demonstration of the `Text` class:

In [2]:
p = Text(p_type='test', default_style='normal')

p.append('Dit is ')
p.append('een test', 'test')
p.append('.')

# str(p) returns concatenated string
print(p)
# repr(p) returns class name, p_type and str(p)
print(repr(p))
# list(p) returns the list of tuples
print(list(p))
# a list comprehension also works
print([e for e in p])

Dit is een test.
<Text 'test' 'Dit is een test.'>
[('Dit is ', 'normal'), ('een test', 'test'), ('.', 'normal')]
[('Dit is ', 'normal'), ('een test', 'test'), ('.', 'normal')]


## Importing the texts

First we import some more useful libraries, and set logging level to DEBUG to make sure we see all logging messages.

In [3]:
import re
import collections
import pathlib
import logging
import unicodedata

from IPython.display import display, HTML
from tf.fabric import Fabric
import pandas as pd

logging.getLogger().setLevel(logging.DEBUG) # for terminal messages

Assuming that the subdirectory `texts` contains the HTML files generated earlier, we can import all files in the pattern `texts/*.html`. At this point we just want to do language statistics and not look at the actual texts, so it is sufficient to import the paragraphs of all texts in no particular order.

In [4]:
files_barwar = pathlib.Path.cwd().glob('texts/bar text *.html') # get source texts
files_urmi_c = pathlib.Path.cwd().glob('texts/cu *.html')

We also prepare a dictionary with some characters that need to be replaced.

In [5]:
# Characters to be replaced
replace = {
    '\u2011': '\u002d',  # U+2011 NON-BREAKING HYPHEN -> U+002D HYPHEN-MINUS
    '\u01dd': '\u0259',  # U+01DD LATIN SMALL LETTER TURNED E -> U+0259 LATIN SMALL LETTER SCHWA
    '\uf1ea': '\u003d',  # U+F1EA Deprecated SIL character -> U+003D '=' EQUALS SIGN
    '\u2026': '...',  # U+2026 '…' HORIZONTAL ELLIPSIS -> three dots
    'J\u0335': '\u0248',  # 'J' + U+0335 COMBINING SHORT STROKE OVERLAY -> U+0248 'Ɉ' LATIN CAPITAL LETTER J WITH STROKE
    'J\u0336': '\u0248',  # 'J' + U+0336 COMBINING LONG STROKE OVERLAY -> U+0248 'Ɉ' LATIN CAPITAL LETTER J WITH STROKE
    '\u002d\u032d': '\u032d\u002d',  # Switch positions of Hyphen and Circumflex accent below
    '\u2011\u032d': '\u032d\u002d',  # Switch positions of Non-breaking hyphen and Circumflex accent below
}

Now we go right ahead to loop over the html files and convert them to a TextFabric structure.

In [6]:
def combine_chars(text):
    """Yield letters combined with combining diacritics"""
    
    char = [] # compose string here: letter + diacritic
    
    for c in text:
        
        # add diacritic
        # indicated as 'Mn': non-spacing combining mark
        if unicodedata.category(c) == 'Mn':
            char.append(c)
            continue
        
        # yield the string; at this point will be: letter + (diacritic)
        if char:
            yield ''.join(char)
            
        char = [c] # save in list and get diacritic next if there is one
        
    yield ''.join(char) # yield empty chars

# keep object counts
raw_features = collections.defaultdict(lambda:collections.defaultdict(set))
raw_oslots = collections.defaultdict(lambda:collections.defaultdict(set))

# initialize counters (will be increased to start from 1)
this_dialect = 0
this_text = 0
this_paragraph = 0
this_line = 0
this_sentence = 0
this_subsentence = 0
this_word = 0
this_morpheme = 0
this_foreign = 0
this_prosa = 0

slot = 0 # i.e. chars

process_dialects = {'Barwar': files_barwar,
                    'Urmi_C': files_urmi_c}

text_ids = []

for dialect, files in process_dialects.items():
    
    # TODO At this point record book/publication/dialect?
    # E.g. SSLL_2016_Urmi_C, HOS_2008_Barwar?
    
    # add dialect object
    this_dialect += 1
    raw_features['dialect'][this_dialect] = dialect
    
    for file in files:
        
        logging.info(f'Processing file {file.name} ...')
        
        for p in html_to_text(file, replace=replace):
            # metadata:
            # - dialect
            # - file.name
            
            if p.type.startswith('gp-') and str(p).strip():
                # store metadata from headings:
                # - text_id
                # - title
                # - informant
                # - place
                # - version (if applicable -- only Urmi_C A35)
                if p.type.startswith('gp-sectionheading'):
                    metadata = {}
                for k, v in parse_metadata(p):
                    metadata[k] = v
            
            elif p.type == 'p':
                # regular paragraphs
                
                # format a text_id with version added (if there is one)
                version = metadata.get('version', '')
                version = f'.{version[-1]}' if version else ''
                text_id = metadata.get('text_id', '') + version
                
                # first check if we need to update metadata
                # informant and place are also added as features of text
                if (metadata
                    and (not raw_features['text_id']
                         or raw_features['text_id'][this_text] != text_id)):
                        
                        
                    text_ids.append(text_id)
                        
                    this_text += 1
                    raw_features['text_id'][this_text] = text_id
                    raw_features['title'][this_text] = metadata['title']
                    raw_features['informant'][this_text] = metadata['informant']
                    raw_features['place'][this_text] = metadata['place']
                    raw_features['dialect'][this_text] = dialect
                    raw_features['filename'][this_text] = file.name
                
                # increment paragraph
                this_paragraph += 1
                
                # start paragraph with an empty marker stack
                marker_stack = []
                
                # set end-of-unit markers to True at the beginning of paragraph,
                # so the units can be increased on encounter of first word character
                sentence_end = True
                subsentence_end = True
                word_end = True
                morpheme_end = True
                foreign_end = True
                prosa_end = True
                
                for text, text_style in p:
                    
                    if text_style == 'verse_no':
                        this_line += 1
                        raw_features['line'][this_line] = text.strip(' ()') # TODO int()?
                        metadata['verse_no'] = text.strip(' ()')  # TODO Remove from metadata dict?
                        continue
                        
                    elif text_style == 'fn_anchor':
                        # TODO handle footnotes in some way, discard for now
                        continue
                    
                    elif text_style == 'comment':
                        continue  # TODO handle comments
                    
                    elif text_style == 'marker':
                        if marker_stack and marker_stack[-1] == text:
                            marker_stack.pop()
                        else:
                            marker_stack.append(text)
                        continue
                    
                    elif text_style not in ('', 'foreign'):
                        logging.debug(f'Unhandled text_style: {repr(text_style)}, {repr(text)}')
                        continue
                    
                    elif text_style == 'foreign' and foreign_end:
                        foreign_end = False
                        if marker_stack:
                            language = marker_stack[-1]
                        else:
                            language = ''
                    
                    else: # text_style == '':
                        if not foreign_end:
                            foreign_end = True
                        pass
                    
                    if (text_style == '' and marker_stack
                        and any(c.isalpha() for c in text)
                        and not text.isalpha()):
                        # In one case, there is no closing marker tag, so force closing the marker
                        # Urmi_C A42 9: 'RzdànyəlaR' (p.154, r.28) 'zdàny' roman, 'əla' cursive
                        # Urmi_C A43 17: 'ʾe-Rbuk̭ḗṱ' (p. 174, r.14), no closing 'R'
                        # Urmi_C B2 16: 'Pʾafšɑ̄rī̀P' (p.250 r.17), inital 'ʾ' cursive
                        marker = marker_stack.pop()
                        logging.warning(f'Unfinished marker: {repr(marker)}, closed forcibly..')
                        logging.debug(f'{dialect}, {metadata["text_id"]}:{metadata["verse_no"]}')
                        logging.debug(f'Text: {repr(text)}')
                    
                    # If we got this far, we have a text string,
                    # with either text_style '' or 'foreign'.
                    # We will iterate over them character by character.
                    for c in combine_chars(text):
                        
                        if c[0].isalpha() or c == '+':
                            
                            # Increment text units on start of new word
                            if morpheme_end:
                                this_morpheme += 1
                                morpheme_end = False
                            if word_end:
                                this_word += 1
                                word_end = False
                            if subsentence_end:
                                this_subsentence += 1
                                subsentence_end = False
                            if sentence_end:
                                this_sentence += 1
                                sentence_end = False
                            if prosa_end:
                                this_prosa += 1
                                prosa_end = False
                            
                            slot += 1
                            
                            pretty_c = unicodedata.normalize('NFC', c) # make pretty utf8 char text
                                      
                            raw_features['utf8'][slot] = pretty_c
                            # initialize 'trailer' feature as empty string,
                            # so we can add characters with '+' operator later
                            raw_features['trailer'][slot] = ''
                                      
                            # add foreign feature here
                            if not foreign_end:
                                raw_features['language'][slot] = language
                            
                            raw_oslots['dialect'][this_dialect].add(slot)
                            raw_oslots['text'][this_text].add(slot)
                            raw_oslots['paragraph'][this_paragraph].add(slot)
                            raw_oslots['line'][this_line].add(slot)
                            raw_oslots['sentence'][this_sentence].add(slot)
                            raw_oslots['subsentence'][this_subsentence].add(slot)
                            raw_oslots['prosa'][this_prosa].add(slot)
                            if not word_end:
                                raw_oslots['word'][this_word].add(slot)
                            if not morpheme_end:
                                raw_oslots['morpheme'][this_morpheme].add(slot)
                        
                        else:  # if c is anything but a letter or '+':
                            if slot == 0:
                                continue  # discard anything before first word character
                            if not morpheme_end:
                                morpheme_end = True
                            if c == '|':
                                prosa_end = True
                                c = '\u02c8'
                            if c not in ('-', '=') and not word_end:
                                word_end = True
                            if c == ',' and not subsentence_end:
                                subsentence_end = True
                            if c in ('.', '!', '?') and not sentence_end:
                                subsentence_end = True
                                sentence_end = True
                            
                            raw_features['trailer'][slot] += c
                
            else:
                logging.debug(f'Unhandled paragraph type: {repr(p.type)}.')
                logging.debug(f'Text: {repr(str(p))}.')

INFO:root:Processing file bar text a15-A17.html ...
INFO:root:Processing file bar text A45.html ...
INFO:root:Processing file bar text a28.html ...
INFO:root:Processing file bar text A49.html ...
INFO:root:Processing file bar text a24.html ...
DEBUG:root:Unhandled paragraph type: 'footer'.
DEBUG:root:Text: ' 7 '.
INFO:root:Processing file bar text A42-A44.html ...
INFO:root:Processing file bar text a25.html ...
INFO:root:Processing file bar text a48.html ...
DEBUG:root:Unhandled paragraph type: 'footer'.
DEBUG:root:Text: ' 1 '.
INFO:root:Processing file bar text a29.html ...
INFO:root:Processing file bar text A9-A13.html ...
INFO:root:Processing file bar text a46-A47.html ...
INFO:root:Processing file bar text A37-A40.html ...
INFO:root:Processing file bar text a18.html ...
INFO:root:Processing file bar text a1-A7.html ...
DEBUG:root:Unhandled paragraph type: 'sdfootnote1'.
DEBUG:root:Text: ' 1 The name Čuxo means ‘one who wears the woolen čuxa garment’. '.
INFO:root:Processing file ba

### Reindex Objects Above Slot Levels

We have given all objects a preliminary node number. Now those node numbers will be renumbered starting from the max slot number.

In [7]:
otype2feature = {
    'dialect':{'dialect'},
    'text': {'text_id', 'title', 'dialect', 'filename', 'informant', 'place'},
    'paragraph': {},
    'line': {'line'},
    'sentence': {},
    'subsentence': {},
    'word': {'trailer'},
    'morpheme': {},
    'prosa': {},
}

onode = max(raw_features['utf8']) # max slot, incremented +1 in loop
node_features = collections.defaultdict(lambda:collections.defaultdict())
edge_features = collections.defaultdict(lambda:collections.defaultdict(set)) # oslots will go here

# first add slot features
# object features must then be added with otype2feature
node_features['utf8'] = raw_features['utf8']
node_features['trailer'] = raw_features['trailer']
node_features['language'] = raw_features['language']

# add slot object types
for slot in node_features['utf8']:
    node_features['otype'][slot] = 'char'    
    
# for objects above slot level, 
# assign new node number and link to feature
for otype in raw_oslots.keys():
    for oID, slots in raw_oslots[otype].items():
        
        # make new object node number
        onode += 1
        node_features['otype'][onode] = otype
        
        # remap node features to node number
        for feat in otype2feature[otype]:
            node_features[feat][onode] = raw_features[feat][oID]
        edge_features['oslots'][onode] = raw_oslots[otype][oID]

The following features are logged:

In [8]:
node_features.keys()

dict_keys(['utf8', 'trailer', 'language', 'otype', 'dialect', 'title', 'text_id', 'informant', 'place', 'filename', 'line'])

The following edges are logged:

In [9]:
edge_features.keys()

dict_keys(['oslots'])

### Purge Old TF Data

In [10]:
for file in pathlib.Path.cwd().glob('tf/?*.tf'):
    file.unlink()

### Save New TF Data

In [11]:
otext = {
    'sectionTypes': 'dialect,text,line',
    'sectionFeatures': 'dialect,title,line',
    'fmt:text-orig-full': '{utf8}{trailer}',
    }

mastermeta = {'author': 'Geoffrey Khan, Cody Kingham, and Hannes Vlaardingerbroek'}

meta = {'':mastermeta,
        'oslots':{'edgeValues':False, 'valueType':'int'},
        'otype':{'valueType':'str'},
        'paragraph':{'valueType':'str'},
        'line':{'valueType':'str'},
        'utf8':{'valueType':'str'},
        'text_id':{'valueType':'str'},
        'title':{'valueType':'str'},
        'dialect':{'valueType':'str'},
        'filename':{'valueType':'str'},
        'language':{'valueType':'str'},
        'trailer':{'valueType':'str'},
        'informant':{'valueType':'str'},
        'place':{'valueType':'str'},
        'otext':otext
       }

TFs = Fabric(locations=['tf/'])

TFs.save(nodeFeatures=node_features, edgeFeatures=edge_features, metaData=meta)

This is Text-Fabric 7.8.5
Api reference : https://annotation.github.io/text-fabric/Api/Fabric/

0 features found and 0 ignored
  0.00s Warp feature "otype" not found in
tf//
  0.01s Warp feature "oslots" not found in
tf//
  0.01s Warp feature "otext" not found. Working without Text-API

  0.00s Exporting 11 node and 1 edge and 2 config features to tf/:
  0.00s VALIDATING oslots feature
  0.09s VALIDATING oslots feature
  0.09s maxSlot=     551014
  0.09s maxNode=     845338
  0.14s OK: oslots is valid
   |     0.00s T dialect              to tf
   |     0.00s T filename             to tf
   |     0.00s T informant            to tf
   |     0.01s T language             to tf
   |     0.01s T line                 to tf
   |     0.23s T otype                to tf
   |     0.00s T place                to tf
   |     0.00s T text_id              to tf
   |     0.00s T title                to tf
   |     0.80s T trailer              to tf
   |     0.71s T utf8                 to tf
   |     

True

## Load New TF Resource for Enhancements

In [12]:
TF = Fabric(locations='tf/')

This is Text-Fabric 7.8.5
Api reference : https://annotation.github.io/text-fabric/Api/Fabric/

14 features found and 0 ignored


In [13]:
N = TF.load('''

text_id paragraph line utf8 
otype title dialect language trailer

''')

N.makeAvailableIn(globals())
print()

  0.00s loading features ...
   |     0.29s T otype                from tf
   |     4.66s T oslots               from tf
   |     0.00s No structure info in otext, the structure part of the T-API cannot be used
   |     1.12s T utf8                 from tf
   |     0.00s T title                from tf
   |     0.84s T trailer              from tf
   |     0.01s T line                 from tf
   |     0.00s T dialect              from tf
   |      |     0.21s C __levels__           from otype, oslots, otext
   |      |     9.28s C __order__            from otype, oslots, __levels__
   |      |     0.37s C __rank__             from otype, __order__
   |      |     8.45s C __levUp__            from otype, oslots, __rank__
   |      |     2.71s C __levDown__          from otype, __levUp__, __rank__
   |      |     2.61s C __boundary__         from otype, oslots, __rank__


   |      |     0.01s c __sections__         WARNING:    1 x section line without containing text


   |      |     0.01s C __sections__         from otype, oslots, otext, __levUp__, __levels__, dialect, title, line
   |     0.00s T text_id              from tf
   |     0.01s T language             from tf
    31s All features loaded/computed - for details use loadLog()



## Enhancements and Extensions

Some features are easier to make once the TF resource is built. The following features will be constructed:

* `utf8` will be extended to word objects to aid in word searches.
* `trans_full` will be added to char, morpheme, and word objects.
* `trans_lite` will be added to char, morpheme, and word objects. Same as `trans_full` but without accents over vowels. 
* `trans_trailer` is added to char, morpheme, and word objects. The trailer is a simple space separation for end of objects.

In [14]:
# Two tables are presented below
# they are largely identical, but edits
# are made as necessary on a char by char basis.
# The tables are used by the function `trans` to 
# to transcribe the utf8 texts

trans_full = {
    # non-latin vowels
    '\u0131': '1',  # 0x0131 ı dotless i
    '\u0251': '@',  # 0x0251 ɑ alpha
    '\u0259': '3',  # 0x0259 ə schwa
    '\u025B': '$',  # 0x025B ɛ open e
    # vowel accents
    '\u0300': '`',  # 0x0300 à grave
    '\u0301': "'",  # 0x0301 á acute
    '\u0304': '_',  # 0x0304 ā macron
    '\u0306': '%',  # 0x0306 ă breve
    '\u0308': '"',  # 0x0308 ä diaeresis
    '\u0303': '~',  # 0x0303 ã tilde
    '\u02C8': '', # 0x2c8 ˈ small vertical line
    # non-latin consonants
    '\u00F0': '6',  # 0x00F0 ð eth
    '\u025F': '&',  # 0x025F ɟ small dotless j with stroke
    '\u0248': '!',  # 0x0248 Ɉ capital J with stroke
    '\u03B8': '8',  # 0x03B8 θ greek theta
    '\u02B8': '7',  # 0x02B8 ʸ small superscript y
    '\u02BE': '}',  # 0x02BE ʾ right half ring (alaph)
    '\u02BF': '{',  # 0x02BF ʿ left half ring (ayin)
    # consonant diacritics
    '\u030C': '<',  # 0x030C x̌ caron
    '\u0302': '^',  # 0x0302 x̂ circumflex
    '\u0307': ';',  # 0x0307 ẋ dot above
    '\u0323': '.',  # 0x0323 x̣ dot below
    '\u032D': '>',  # 0x032D x̭ circumflex below
}

trans_lite = {
    # non-latin vowels
    '\u0131': '1',  # 0x0131 ı dotless i
    '\u0251': 'a',  # 0x0251 ɑ alpha
    '\u0259': '3',  # 0x0259 ə schwa
    '\u025B': 'e',  # 0x025B ɛ open e
    # vowel accents
    '\u0300': '',  # 0x0300 à grave
    '\u0301': "",  # 0x0301 á acute
    '\u0304': '',  # 0x0304 ā macron
    '\u0306': '',  # 0x0306 ă breve
    '\u0308': '',  # 0x0308 ä diaeresis
    '\u0303': '',  # 0x0303 ã tilde
    '\u02C8': '', # 0x2c8 ˈ small vertical line
    # non-latin consonants
    '\u00F0': '6',  # 0x00F0 ð eth
    '\u025F': '&',  # 0x025F ɟ small dotless j with stroke
    '\u0248': '!',  # 0x0248 Ɉ capital J with stroke
    '\u03B8': '8',  # 0x03B8 θ greek theta
    '\u02B8': '7',  # 0x02B8 ʸ small superscript y
    '\u02BE': '}',  # 0x02BE ʾ right half ring (alaph)
    '\u02BF': '{',  # 0x02BF ʿ left half ring (ayin)
    # consonant diacritics
    '\u030C': '<',  # 0x030C x̌ caron
    '\u0302': '^',  # 0x0302 x̂ circumflex
    '\u0307': ';',  # 0x0307 ẋ dot above
    '\u0323': '.',  # 0x0323 x̣ dot below
    '\u032D': '>',  # 0x032D x̭ circumflex below
}

def trans(s, table):
    '''
    Transcribes a text.
    '''
    s = unicodedata.normalize('NFD', s)
    s = s.strip('\n |,.!?:;')
    return ''.join([table.get(c, c) for c in s])

## Remake Existing Char Features and Extend them with New ones

In [15]:
extend_features = collections.defaultdict(lambda: collections.defaultdict())

# first re-create the features
for char in F.otype.s('char'):
    # remake utf8
    extend_features['utf8'][char] = F.utf8.v(char)
    # remake trailer
    extend_features['trailer'][char] = F.trailer.v(char)
    
    # apply extra features
    extend_features['trans_full'][char] = trans(F.utf8.v(char), trans_full)
    extend_features['trans_lite'][char] = trans(F.utf8.v(char), trans_lite)

### Extend `utf8` Feature to Word and Morpheme Objects

In [16]:
# add words and morphemes; compose without trailer

for otype in ('word', 'morpheme'):
    
    for obj in F.otype.s(otype):
    
        # compose text features from characters
        utf8 = '' # full utf8 representation
        transF = '' # trans full
        transL = '' # trans lite
        
        for char in L.d(obj, 'char'):
            
            utf8 += F.utf8.v(char)
            transF += trans(F.utf8.v(char), trans_full)
            transL += trans(F.utf8.v(char), trans_lite)
            
            # only add trailing morpheme hyphens to words
            if otype == 'word' and F.trailer.v(char) == '-':
                utf8 += F.trailer.v(char)
                
            # add trailer feature
            if char == L.d(obj, 'char')[-1]: # is last char
                extend_features['trailer'][obj] = F.trailer.v(char)
    
        # add all features to objects
        pretty_txt = unicodedata.normalize('NFC', utf8) # make pretty unicode 
        extend_features['utf8'][obj] = pretty_txt
        extend_features['trans_full'][obj] = transF
        extend_features['trans_lite'][obj] = transL

## Export Extended Features

In [17]:
# make a new otext to include new text formats
otext = {
    'sectionTypes': 'dialect,text,line',
    'sectionFeatures': 'dialect,title,line',
    'fmt:text-orig-full': '{utf8}{trailer}',
    'fmt:text-trans-full': '{trans_full}{trailer}',
    'fmt:text-trans-plain': '{trans_lite}{trailer}'
    }

meta = {'':mastermeta,
        'utf8': {'valueType':'str'},
        'trans_full':{'valueType':'str',
                      'description':'A full transcription on char, morpheme, and word objects.'},
        'trans_lite':{'valueType':'str',
                      'description':'A lite transcription on char, morpheme, and word objects.'},
        'trailer':{'valueType':'str'},
        'otext': otext
       }

TFs = Fabric(locations=['tf/'])

TFs.save(nodeFeatures=extend_features, metaData=meta)

This is Text-Fabric 7.8.5
Api reference : https://annotation.github.io/text-fabric/Api/Fabric/

14 features found and 0 ignored
  0.00s Exporting 4 node and 0 edge and 1 config features to tf/:
   |     0.96s T trailer              to tf
   |     0.96s T trans_full           to tf
   |     0.96s T trans_lite           to tf
   |     1.00s T utf8                 to tf
   |     0.00s M otext                to tf
  3.88s Exported 4 node features and 0 edge features and 1 config features to tf/


True

## Re-Load Enhanced TF Resource

In [17]:
TF = Fabric(locations='tf/')

This is Text-Fabric 7.8.4
Api reference : https://annotation.github.io/text-fabric/Api/Fabric/

14 features found and 0 ignored


In [18]:
N = TF.load('''

text_id paragraph line utf8 
otype title dialect language trailer

''')

N.makeAvailableIn(globals())
print()

  0.00s loading features ...
   |     0.00s No structure info in otext, the structure part of the T-API cannot be used
   |     1.79s T utf8                 from tf
   |     1.19s T trailer              from tf
  4.52s All features loaded/computed - for details use loadLog()



### Find Source of Line Warning

In [19]:
for line in F.otype.s('line'):
    
    if not L.u(line, 'text'):
        
        print(line, T.text(line))
        print()
        print(L.d(line, 'word'))

552511 mə́ra wírra šəkwánta qamɛ́θa šqilála ða-ḥábba ʾu-plìṭla.ˈ wírra t-tə́rte šqilála ða-ḥábba-w plìṭla.ˈ wírra t-ṭə́ḷḷəθ ða-ḥábba-w plìṭla.ˈ mə́ra ha-t-xásli ʾan-xə̀ṭṭe,ˈ ʾan-šəkwáne ṱ-áwɛ ðà-ða wára.ˈ líθ dúka t-tə̀rte.ˈ har-ðà-ða wára.ˈ mə́re dū̀s,ˈ ʾáy čú-ga la-xàsla.ˈ ʾìθwaˈ xá familyˈ bàbaˈ ʾu-brònaˈ ʾu-báxtət bròna.ˈ ʾo-bàbaˈ píšɛwa sàwa.ˈ sáwɛwa gu-bɛ̀θa.ˈ kálθe díye sqìdla mə́nne.ˈ

(657410, 657411, 657412, 657413, 657414, 657415, 657416, 657417, 657418, 657419, 657420, 657421, 657422, 657423, 657424, 657425, 657426, 657427, 657428, 657429, 657430, 657431, 657432, 657433, 657434, 657435, 657436, 657437, 657438, 657439, 657440, 657441, 657442, 657443, 657444, 657445, 657446, 657447, 657448, 657449, 657450, 657451, 657452, 657453, 657454, 657455, 657456, 657457, 657458)


In [1]:
hex(ord('ˈ'))

'0x2c8'

In [20]:
for w in L.d(552509, 'word'):
    text = L.u(w, 'text')[0]
    print(w, F.dialect.v(text), T.sectionFromNode(w), T.text(w))

657361 Barwar ('Barwar', 'THE STORY WITH NO END', '3') mə́ra 
657362 Barwar ('Barwar', 'THE STORY WITH NO END', '3') ʾíθwa 
657363 Barwar ('Barwar', 'THE STORY WITH NO END', '3') xa-málka 
657364 Barwar ('Barwar', 'THE STORY WITH NO END', '3') ẓalə́m-wewa 
657365 Barwar ('Barwar', 'THE STORY WITH NO END', '3') maxwàθux.ˈ 
657366 Barwar ('Barwar', 'THE STORY WITH NO END', '3') ʾíθwale 
657367 Barwar ('Barwar', 'THE STORY WITH NO END', '3') xa-muqaṭăʿa 
657368 Barwar ('Barwar', 'THE STORY WITH NO END', '3') rába 
657369 Barwar ('Barwar', 'THE STORY WITH NO END', '3') gòṛa.ˈ 
657370 Barwar ('Barwar', 'THE STORY WITH NO END', '3') mjŭmɛ́le 
657371 Barwar ('Barwar', 'THE STORY WITH NO END', '3') xə́ṭṭe 
657372 Barwar ('Barwar', 'THE STORY WITH NO END', '3') t-kúlla 
657373 Barwar ('Barwar', 'THE STORY WITH NO END', '3') d-ɛ-mànṭăqa.ˈ 
657374 Barwar ('Barwar', 'THE STORY WITH NO END', '3') drɛ́le 
657375 Barwar ('Barwar', 'THE STORY WITH NO END', '3') gu-sèylo.ˈ 
657376 Barwar ('Barwar', 'TH

Note that this line extends over two texts. This is due to an error in the source text, where there is a missing "(1)"

### Test Text Features

#### Sectioning

In [21]:
T.sectionFromNode(1)

('Barwar', 'The Monk And The Angel', '1')

### UTF8 Feature

In [22]:
utf8s = {'first_chars' : list(F.otype.s('char'))[:20],
         'first_morphs': list(F.otype.s('morpheme'))[:15],
         'first_words' : list(F.otype.s('word'))[:10],}

for otype, objs in utf8s.items():
    print(otype)
    for obj in objs:
        print(f'\t{F.utf8.v(obj)}')
    print()

first_chars
	ʾ
	ì
	θ
	w
	a
	x
	a
	r
	à
	b
	b
	ə
	n
	t
	í
	w
	ɛ
	w
	a
	g

first_morphs
	ʾìθwa
	xa
	ràbbən
	tíwɛwa
	gu
	xa
	gəppìθa
	θéle
	xa
	náša
	swarìya
	rakáwa
	ṣléle
	rəš
	xa

first_words
	ʾìθwa
	xa-ràbbən
	tíwɛwa
	gu-xa-gəppìθa
	θéle
	xa-náša
	swarìya
	rakáwa
	ṣléle
	rəš-xa-ʾɛ̀na



### Trailer

Object + trailer is shown.

In [23]:
for otype, objs in utf8s.items():
    print(otype)
    for obj in objs:
        otext = F.utf8.v(obj)
        print(f'\t{otext}{F.trailer.v(obj)}')
    print()

first_chars
	ʾ
	ì
	θ
	w
	aˈ 
	x
	a-
	r
	à
	b
	b
	ə
	n,ˈ 
	t
	í
	w
	ɛ
	w
	a 
	g

first_morphs
	ʾìθwaˈ 
	xa-
	ràbbən,ˈ 
	tíwɛwa 
	gu-
	xa-
	gəppìθa.ˈ 
	θéle 
	xa-
	náša 
	swarìya,ˈ 
	rakáwa.ˈ 
	ṣléle 
	rəš-
	xa-

first_words
	ʾìθwaˈ 
	xa-ràbbən,ˈ 
	tíwɛwa 
	gu-xa-gəppìθa.ˈ 
	θéle 
	xa-náša 
	swarìya,ˈ 
	rakáwa.ˈ 
	ṣléle 
	rəš-xa-ʾɛ̀na.ˈ 



## Statistics

### Object Types and Counts

In [24]:
for otype in F.otype.all:
    print('{:20}{:>10}'.format(otype, len(list(F.otype.s(otype)))))

dialect                      2
text                       126
paragraph                  465
line                      2543
sentence                 16784
subsentence              24541
prosa                    35967
word                     93762
morpheme                120134
char                    551014


In [25]:
print('texts and their word counts: \n')

for text in F.otype.s('text'):
    
    text_words = L.d(text, 'word')
    text_morphemes = L.d(text, 'morpheme')
    
    print(text, F.text_id.v(text), F.dialect.v(text), F.title.v(text))
    print(f'\t{len(text_words)} words, {len(text_morphemes)} morphemes')

texts and their word counts: 

551017 A15 Barwar The Monk And The Angel
	680 words, 950 morphemes
551018 A16 Urmi_C THE MONK WHO WANTED TO KNOW WHEN HE WOULD DIE
	368 words, 487 morphemes
551019 A17 Barwar THE WISE YOUNG MAN
	1091 words, 1482 morphemes
551020 A45 Barwar THE FOX AND THE STORK
	70 words, 102 morphemes
551021 A28 Barwar THE TALE OF RUSTAM (1)
	1008 words, 1316 morphemes
551022 A49 Barwar THE CROW AND THE CHEESE
	51 words, 71 morphemes
551023 A24 Barwar THE TALE OF PARIZADA, WARDA AND NARGIS
	2016 words, 2698 morphemes
551024 A42 Barwar THE FOX AND THE LION
	95 words, 124 morphemes
551025 A43 Barwar SOUR GRAPES
	62 words, 82 morphemes
551026 A44 Barwar THE CAT AND THE MICE
	99 words, 138 morphemes
551027 A25 Barwar THE TALE OF FARXO AND SƏTTIYA
	2490 words, 3303 morphemes
551028 A48 Barwar THE MAN WHO CRIED WOLF
	199 words, 250 morphemes
551029 A29 Barwar THE TALE OF RUSTAM (2)
	1645 words, 2254 morphemes
551030 A9 Barwar THE SCORPION AND THE SNAKE
	216 words, 298 morpheme

In [26]:
text = F.otype.s('text')[0]

In [27]:
len(L.d(text, 'word'))

680

In [28]:
print(F.title.v(text))

for sent in F.otype.s('line')[:10]:
    print(sent, T.text(sent))


The Monk And The Angel
551608 ʾìθwaˈ xa-ràbbən,ˈ tíwɛwa gu-xa-gəppìθa.ˈ θéle xa-náša swarìya,ˈ rakáwa.ˈ ṣléle rəš-xa-ʾɛ̀na.ˈ tìwle,ˈ xílle mə̀ndi,ˈ štéle mìya.ˈ ʾíθwale xákma zùze.ˈ qímle šqilìle.ˈ muttìleˈ rəš-d-ɛ-ʾɛ̀na.ˈ ʾàwwaˈ munšìle zúze díye.ˈ zìlle.ˈ ʾáwwa zílle b-ʾùrxa.ˈ
551609 θéle xá rakáwa xèna,ˈ swarìya.ˈ zílle rəš-ʾɛ̀na.ˈ qəm-xazèlaˈ ʾə̀mma dináre.ˈ šqilíle jal-jàldeˈ muttíle gu-jɛ̀beˈ ʾu-zìlle.ˈ ʾo-qamàyaˈ ʾámər ʾòhˈ zúzi munšìli.ˈ qɛ́mən dɛ̀ṛənˈ ʾázən šáqlən zùziˈ m-rəš-ʾɛ̀na.ˈ
551610 ha-t-ʾáθe ʾo-náša qamàyaˈ máṭe l-ʾɛ̀naˈ ʾáθe xa-náša sàwa.ˈ máṭe rəš-ʾɛ̀naˈ mattúla kàrteˈ ʾu-tíwle manyòxe.ˈ ʾo-qamáya ṱ-íle zúze mùnšyaˈ θéle ʾə́lle dìye.ˈ ʾàmərˈ mpáləṭla zùzi!,ˈ ʾə́mma dináre ʾána hon-mùnšəlla láxxa.ˈ lázəm yawə̀tla.ˈ yába lán-xəzya zùze,ˈ lá ʾáxxa-w tàmmaˈ ʾu-kízle b-ay-gòta.ˈ là.ˈ
551611 mə́re qaṭlə̀nnux.ˈ mə́re qṭùl!ˈ lìtli.ˈ zúze làn-xəzya.ˈ qìmleˈ qəm-qaṭə̀lle.ˈ qəm-qaṭə̀lle.ˈ ràbbənˈ yăðət-mà-yle?ˈ ràbbənˈ ʾáwwa ṱ-i-sàxəð l-ʾálahaˈ ʾu-ṱ-i-mṣàle-uˈ lé-y-ʾaxəl bə̀sr

In [29]:
for w in L.d(551615, 'word'):
    print(w, F.utf8.v(w))

631644 šlàma-llɛxu
631645 b-šɛ́na
631646 b-ṭawàθa
631647 páqðu
631648 tìwe
631649 tíwela
631650 tàma
631651 ʾu-xílela
631652 ʾixàla
631653 ʾu-píše
631654 dmìxe
631655 dmìxela
631656 b-lɛ̀le
631657 maláxa
631658 mŭṛə̀šle
631659 qa-ràbbən
631660 mə́re
631661 qù
631662 qù
631663 mə́re
631664 há
631665 də-šúqlən
631666 dàmxəx
631667 lɛ́t-mira
631668 ṱ-óðən
631669 b-xàbrux
631670 qu-šqúlla
631671 ʾáyya
631672 skìnta
631673 si-prúmle
631674 ʾáwwa
631675 yála
631676 zòra
631677 ṱ-íle
631678 gu-dudìya


### Testing Query Capability

In [30]:
w = F.utf8.v(631728)

find = list(S.search(f'''

word utf8={w}

'''))

len(find)

1